In [1]:
import pandas as pd
import numpy as np
from yahooquery import Ticker

In [2]:
from datetime import datetime
now = datetime.now()
today_datetime = now.strftime("%m%d%Y_%H%M%S")

In [3]:
import sys
import os.path
from os import path

In [4]:
if sys.argv[1] != 0:
    
    # filename = sys.argv[1]
    filename = "stock_list"
    excel_filename = filename + ".xlsx"
    if not path.exists(excel_filename):
        print(f"The Excel file name {excel_filename} you entered does not exist. Try again!")
        exit()
else: 
    print("You need to enter a file name (no file type extension is required - it is default to xlsx)!")
    exit()

In [5]:
extra_columns = ['sharePortfolio', 'wBeta','avgBeta','marketValue', 'change', '%return', 'total', 'trailingPE', 'forwardPE','trailingEps', 'forwardEps', 'dividendRate', 'recommend', 'recommendTrend' ]

ignore_stock_substr = '**'

In [6]:
df_xl = pd.ExcelFile(excel_filename)
sheet_names = df_xl.sheet_names

In [7]:
writer = pd.ExcelWriter(filename+ "_" + today_datetime + ".xlsx", engine='xlsxwriter')

In [8]:
def convertRecommendListToStr(recommends):

    liststr=""
    if 'recommendedSymbols' in recommends:
        for list in recommends['recommendedSymbols']:
            sym = list['symbol']
            score = list['score']
            str1= sym + " score: " + str(score)
            str2 = "[" + str1 + "] "
            liststr += str2
        
    return liststr

In [9]:
def convertRecommendTrendListToStr(trend):

    liststr=""
    strong_buy = trend['strongBuy']
    buy = trend['buy']
    hold = trend['hold']
    sell = trend['sell']
    strong_sell = trend['strongSell']
    
    str1= "Strong Buy: " + str(strong_buy) + " Buy: " + str(buy) + " Hold: " + str(hold) + " Sell: " + str(sell) + " Strong Sell: " + str(strong_sell)
    str2 = "[" + str1 + "] "
    liststr += str2
        
    return liststr

In [10]:
def fill_history(i, df, history):
    
    df.at[i, 'open'] = history.iloc[0]['open']
    df.at[i, 'adjclose'] = history.iloc[0]['adjclose']
    
    return df

In [11]:
def fill_mutual_fund_history(i, df, history):
    
    df.at[i, 'open'] = history['meta']['chartPreviousClose']
    df.at[i, 'adjclose'] = history['meta']['regularMarketPrice']
    
    return df

In [12]:
def fill_summary_details(i, df, details):
  
    if 'trailingAnnualDividendRate' in details:
        df.at[i, 'dividendRate'] = details['trailingAnnualDividendRate']
        
    if 'beta' in details:
        df.at[i, 'beta'] = details['beta']
     
    if 'trailingPE' in details:
        df.at[i, 'trailingPE'] = details['trailingPE']
        
    if 'forwardPE' in details:
        df.at[i, 'forwardPE'] = details['forwardPE']
        
    return df
        

In [13]:
def fill_keystats(i, df, keystats):
    
    if 'trailingEps' in keystats:
        df.at[i, 'trailingEps'] = keystats['trailingEps']
        
    if 'forwardEps' in keystats:
        df.at[i, 'forwardEps'] = keystats['forwardEps']
    
    return df

In [14]:
def fill_recommendations(i, df, recommends):
    
    # convert dat to string and fill in the data if the data exists (not a str error statement)
    if not type(recommends) is str:
        df.at[i, 'recommend']  = convertRecommendListToStr(recommends)
    
    return df

In [15]:
def fill_recommendation_trends(i, df, symbol, trends):
    
    if type(trends) is dict:
        if 'trend' in trends[symbol]:
            trend = trends[symbol]['trend'][0]
            df.at[i, 'recommendTrend']  = convertRecommendTrendListToStr(trend)
        else:
            df.at[i, 'recommendTrend'] = ""
            
    else: # data frame
        df.at[i, 'recommendTrend']  = convertRecommendTrendListToStr(trends.loc[symbol].loc[0])
    
    return df

In [16]:
def get_stock_data(df, stock_list):
    
    yf_df = Ticker(stock_list, asynchronous=True)
    
    yf_details = yf_df.summary_detail
    yf_history = yf_df.history(period='1d')
    yf_keystats = yf_df.key_stats
    yf_recommends = yf_df.recommendations
    yf_recommend_trends = yf_df.recommendation_trend
    
    # initialize the  data type to str for recommendation columns
    df['recommend'] = df['recommend'].astype(str)
    df['recommendTrend'] = df['recommendTrend'].astype(str)
   
    for i, symbol in enumerate (df['Symbol']):
        if  ignore_stock_substr not in symbol:
            if type(yf_history) is dict:           
                if (type(yf_history[symbol])) is dict:
                    # check if it is a mutual fund
                    if 'meta' in yf_history[symbol]:
                        fill_mutual_fund_history(i,df, yf_history[symbol]) 
                        
                # there is an error    
                elif (type(yf_history[symbol])) is str:  
                    continue;
                    
            else: # data frame
                fill_history(i,df, yf_history.loc[symbol])            
                               
            fill_summary_details(i,df, yf_details[symbol])
            fill_keystats(i,df, yf_keystats[symbol])
            fill_recommendations(i,df, yf_recommends[symbol]) 
            fill_recommendation_trends(i, df, symbol, yf_recommend_trends)

            if symbol == "SPY": df.at[i, 'beta'] = 1
                
    return df

In [17]:
def get_stock_list(df):

    stock_list = []
    for i, symbol in enumerate (df['Symbol']):
        if ignore_stock_substr not in symbol:
            stock_list.append(symbol)
            
    return stock_list

In [18]:
def build_data(df):
    df['beta'] = df['beta'].fillna(0)
    df['marketValue'] = df['adjclose'] * df['Quantity']
    df['marketValue'] = df['marketValue'].fillna(0)
    df.at[0,'total'] = sum(df['marketValue'])
    df['sharePortfolio'] = df['marketValue'] / df.at[0,'total']
    df['change'] = df['marketValue'] - df['BaseCost']
    df['%return'] = df['change'].div(df['BaseCost']).replace(np.inf, 0) * 100
    df['wBeta'] = df['sharePortfolio'] * df['beta']
    df.at[0,'avgBeta'] = sum(df['wBeta'])
    return df

In [19]:
def build_sheet(df):
    df_ext=pd.DataFrame(columns=extra_columns)
    df = pd.concat([df, df_ext], axis=1)
    stock_list = get_stock_list(df)
    df = get_stock_data(df, stock_list)
    df = build_data(df)
    return df

In [20]:
for i, name in enumerate (sheet_names):
    df = df_xl.parse(sheet_name=i)
    df = build_sheet(df)
    df.to_excel(writer, sheet_name=name, index=False)

In [21]:
writer.save()